# Summary
We map cropland across Africa.
1. Set up environment
2. Import AOI
3. Create a cloudless Sentinel-2 collection
4. Create periods per year

### Future
-

# Set up the environment

In [ ]:
# Import and/or install libraries
import subprocess, os, gcsfs, json

try:
    import geemap, ee
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', '-U', 'geemap'])
    import geemap, ee


In [ ]:
# Connect to Google Drive to access files

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Connect to Google Earth Engine if neccessary

service_account = os.environ.get('GOOGLE_SERVICE_ACCOUNT')
credentials = ee.ServiceAccountCredentials(service_account, os.environ.get('GOOGLE_APPLICATION_CREDENTIALS'))
ee.Initialize(credentials)

# Import AOI

In [ ]:
with open("/content/drive/MyDrive/data/cropland/central_aoi.geojson") as f:
    json_data_aoi = json.load(f)

aoi = geemap.geojson_to_ee(json_data_aoi)

with open("/content/drive/MyDrive/data/cropland/cropland_central.geojson") as f:
    json_data_crop_aoi = json.load(f)

cropland_aoi = geemap.geojson_to_ee(json_data_crop_aoi)


# Create a cloudless Sentinel-2 collection

In [ ]:
CLOUD_FILTER = 80
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50


def get_s2_sr_cld_col(datefilter):
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filter(datefilter)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filter(datefilter)
        )

    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))


def add_cloud_bands(img):
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')
    return img.addBands(ee.Image([cld_prb, is_cloud]))


def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)


def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)


date_filter = ee.Filter.date(ee.Date('2015-12-31'), ee.Date('2023-01-01'))

sentinel = get_s2_sr_cld_col(date_filter).map(add_cld_shdw_mask).map(apply_cld_shdw_mask)

sentinel_vis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2']
}

sentinel_clip = sentinel.filterBounds(aoi)

# Create periods per year

In [ ]:
bands = ['NDVI', 'B5', 'B6', 'B7', 'B8A']

# Calculate NDVI and clip to aoi
def calculate_ndvi(img):
    ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI').clip(aoi)
    return img.addBands(ndvi)

sentinel_ndvi = sentinel_clip.map(calculate_ndvi).select(bands)



In [ ]:
# Function to create median periods

def compute_yearly_median(year_start_date, year_end_date, days):
    num_periods = year_end_date.difference(year_start_date, 'day').divide(days).floor()
    date_sequence = ee.List.sequence(0, num_periods.subtract(1))

    def compute_date(n):
        n = ee.Number(n)
        return year_start_date.advance(n.multiply(days), 'day')

    dates_list = date_sequence.map(compute_date)

    def create_medians(image_index):
        image_index = ee.List(image_index)
        this_date = ee.Date(image_index.get(0))
        period = ee.Number(image_index.get(1))
        median_img = sentinel_ndvi.filterDate(this_date, this_date.advance(days, 'day')).median()
        return median_img.set('period', period)

    return ee.ImageCollection(ee.List(dates_list.zip(date_sequence)).map(create_medians))



In [ ]:
def add_period_property(img_collection):
  num_images = img_collection.size()
  sequence = ee.List.sequence(0, num_images.subtract(1))

  def add_counter(image_index):
      image_index = ee.List(image_index)
      image = ee.Image(image_index.get(0))
      counter = ee.Number(image_index.get(1))
      return image.set('counter', counter)

  new_image_collection = ee.ImageCollection(ee.List(img_collection.zip(sequence)).map(add_counter))
  return new_image_collection

In [ ]:
# Compute the periods
days = 60

y_start = ee.Date('2022-01-01')
y_end = ee.Date('2023-01-01')
y1_start = y_start.advance(-1, 'year')
y1_end = y_end.advance(-1, 'year')
y2_start = y_start.advance(-2, 'year')
y2_end = y_end.advance(-2, 'year')

y_periods = compute_yearly_median(y_start, y_end, days)
y1_periods = compute_yearly_median(y1_start, y1_end, days)
y2_periods = compute_yearly_median(y2_start, y2_end, days)

In [ ]:
# def replace_masked_with_previous_year(feature):
#     primary = ee.Image(feature.get('primary'))
#     secondary = ee.Image(feature.get('secondary'))
#     updated_image = primary.where(primary.mask().Not(), secondary)

#     return updated_image


def replace_masked_with_previous_year(feature):
    primary = ee.Image(feature.get('primary'))
    secondary = ee.Image(feature.get('secondary'))

    # Update the mask of the secondary image to be the inverse of the primary image mask
    updated_secondary = secondary.updateMask(primary.mask().Not())

    # Combine the primary and updated_secondary images
    updated_image = primary.add(updated_secondary)

    return updated_image


In [ ]:
def join_images(feature):
    primary = ee.Image(feature.get('primary'))
    secondary = ee.Image(feature.get('secondary'))
    combined_image = ee.Image.cat(primary, secondary)

    return combined_image


# YEAR 1: Join images
filter_eq = ee.Filter.equals(leftField='period', rightField='period')
inner_join = ee.Join.inner()
inner_joined_1 = inner_join.apply(y_periods, y1_periods, filter_eq)
joined_images_1 = ee.ImageCollection(inner_joined_1.map(join_images))

# YEAR 1: Map the function over the image collection
backed_filled_1 = inner_joined_1.map(replace_masked_with_previous_year)


# YEAR 2: Join images
filter_eq = ee.Filter.equals(leftField='period', rightField='period')
inner_join = ee.Join.inner()
inner_joined_2 = inner_join.apply(backed_filled_1, y2_periods, filter_eq)
joined_images_2 = ee.ImageCollection(inner_joined_2.map(join_images))

# YEAR 2: Map the function over the image collection
backed_filled_2 = inner_joined_2.map(replace_masked_with_previous_year)

In [ ]:
feat = inner_joined_1.first()
combined = replace_masked_with_previous_year(feat)
print(combined.getInfo())

In [ ]:
time_img_y = y_periods.select('NDVI').toBands().clip(aoi)
time_img_y1 = y1_periods.select('NDVI').toBands().clip(aoi)
time_img_filled = backed_filled_1.select('NDVI').toBands().clip(aoi).rename(time_img_y.bandNames())


# Import AOI

In [ ]:
bands = ['0_NDVI', '1_NDVI', '2_NDVI', '3_NDVI', '4_NDVI', '5_NDVI']
image_aoi = time_img_y.select(bands)

# Add metrics

In [ ]:
# sentinel_bands = ['B5', 'B6', 'B7', 'B8A'] # red edge

# Red-edge slope
def calc_slope(img):
    slope = img.select('B7').subtract(img.select('B5')).divide(ee.Number(835.1-703.9)).rename('slope')
    return img.addBands(slope)

sentinel_slope = backed_filled.map(calc_slope)
sentinel_bands = ['slope'] # red edge
red_edge_img = sentinel_slope.select(sentinel_bands).median()


# Forest height
height = ee.ImageCollection('users/potapovpeter/GEDI_V27').median().select('b1').rename('height')


In [ ]:
sentinel_max = backed_filled.select('NDVI').max().rename('max')
sentinel_min = backed_filled.select('NDVI').min().rename('min')

image_bands = image_aoi.addBands(sentinel_max).addBands(sentinel_min).addBands(red_edge_img)

# Sample points

In [ ]:
# Randomly split the data into 70% training and 30% validation
cropland_split = cropland_aoi.randomColumn(seed=5)
training = cropland_split.filter(ee.Filter.lt('random', 0.7))
validation = cropland_split.filter(ee.Filter.gte('random', 0.7))

# Sample the pixel values
training_sampled = image_bands.sampleRegions(collection=training, properties=['Class'], scale=30, tileScale=4)
validation_sampled = image_bands.sampleRegions(collection=validation, properties=['Class'], scale=30, tileScale=4)

In [ ]:
# Classify
classifier = ee.Classifier.smileRandomForest(10).train(features=training_sampled, classProperty='Class', inputProperties=image_bands.bandNames())
classified = image_bands.classify(classifier)

validated = validation_sampled.classify(classifier)
confusionMatrix = validated.errorMatrix('Class', 'classification')

overallAccuracy = confusionMatrix.accuracy()

producersAccuracy = confusionMatrix.producersAccuracy() # Sensitivity (recall)
consumersAccuracy = confusionMatrix.consumersAccuracy() # Specificity (precision)

# confusionMatrix.getInfo()

In [ ]:
print('Accuracy')
print(overallAccuracy.getInfo())

# print('Sensitivity')
# print(producersAccuracy.getInfo())

# print('Specificity')
# print(consumersAccuracy.getInfo())

In [ ]:
cropland = classified.eq(1).selfMask()

# Map

In [ ]:
primary = ee.Image(feat.get('primary'))
secondary = ee.Image(feat.get('secondary'))
clouds = primary.mask().Not()
input = primary.unmask(-9999)

In [ ]:
ndvi_vis = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}

Map = geemap.Map()
Map.add_basemap('SATELLITE')
# Map.addLayer(clouds, {}, 'clouds')
# Map.addLayer(input, {}, 'input')

# Map.addLayer(time_img_y.select('3_NDVI'), ndvi_vis, 'y')
# Map.addLayer(time_img_filled.select('3_NDVI'), ndvi_vis, 'filled_1')
Map.addLayer(primary.select('NDVI'), ndvi_vis, 'y')
Map.addLayer(secondary.select('NDVI'), ndvi_vis, 'y1')
Map.addLayer(combined.select('NDVI'), ndvi_vis, '0_NDVI_new')

Map.setCenter(10.752, 0.698, 12)
Map